In [8]:
import pandas as pd
import os
import time
import langchain
from typing import List,Dict,Any

from langchain.schema import document
from langchain_core.documents import Document
from langchain.text_splitter import (RecursiveCharacterTextSplitter,CharacterTextSplitter,TokenTextSplitter)
from sqlalchemy.testing.suite.test_reflection import metadata

print("Setup completed")


Setup completed


In [13]:
doc=Document(
    page_content="THis is the main text content that will be embedded and searched!",
    metadata={
        "source":"Example.txt",
        "author":"divyansh",
        "page":1,
        "date_created":"2025-01-01",
        "custom_field":"any_val"
    }
)
print("Document Structure")
print(f"metadata:{doc.metadata}")
print(f"page content:{doc.page_content}")

Document Structure
metadata:{'source': 'Example.txt', 'author': 'divyansh', 'page': 1, 'date_created': '2025-01-01', 'custom_field': 'any_val'}
page content:THis is the main text content that will be embedded and searched!


In [16]:
os.makedirs("data/text_files",exist_ok=True)

In [18]:
sample_texts={
    "data/text_files/python_intro.txt":"""
        python programming language is used for building ai applications
    """
}

for filepath,contents in sample_texts.items():
    with open(filepath,"w",encoding='utf-8') as f:
        f.write(contents)

print("sample texts:")
print(sample_texts)

sample texts:
{'data/text_files/python_intro.txt': '\n        python programming language is used for building ai applications\n    '}


### Text-Loader

In [19]:
from langchain_community.document_loaders import TextLoader
# loading a single text file
loader=TextLoader(file_path='data/text_files/python_intro.txt',encoding='utf-8')

In [23]:
documents = loader.load()

In [38]:
print(documents[0].page_content[1:])
print(documents[0].metadata)
print(len(documents))

        python programming language is used for building ai applications
    
{'source': 'data/text_files/python_intro.txt'}
1


In [39]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # regex to match files
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)

documents=dir_loader.load()

100%|██████████| 1/1 [00:00<00:00, 1000.79it/s]


In [46]:
documents[0].page_content.strip()

'python programming language is used for building ai applications'